In [1]:
from lm_polygraph.generation_metrics.openai_fact_check import OpenAIFactCheck
from lm_polygraph.stat_calculators.extract_claims import Claim
import json
import pandas as pd
from tqdm import tqdm
arabic_checker = OpenAIFactCheck('gpt-4o', language='en')

## load claims
full_claims = pd.read_csv('../workdir/arabic_claims.csv')

/home/boda/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/boda/miniforge3/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree

In [2]:
full_claims.columns

Index(['question', 'bio', 'claim', 'sentence', 'tokens'], dtype='object')

In [3]:
for i, row in tqdm(full_claims.iterrows(), total=full_claims.shape[0]):
    claim = [Claim(claim_text=row['claim'], sentence=row['sentence'], aligned_tokens=row['tokens'])]
    check_dict = { "input_texts": [row['question']],
                  "claims": [claim]}
    chatgpt_response = arabic_checker(check_dict, None, None )
    full_claims.at[i,'chatgpt_verdict'] = chatgpt_response[0]



  0%|          | 0/478 [00:00<?, ?it/s]

100%|██████████| 478/478 [43:20<00:00,  5.44s/it]


In [6]:
full_claims['chatgpt_verdict'].value_counts().sum(), len(full_claims)

(471, 478)

In [7]:
full_claims.sample(frac=1).to_csv("../workdir/arabic_fact_checked_claims_two_steps_english_prompt_shuffled.csv", index=False)

full_claims.to_csv("../workdir/arabic_fact_checked_claims_two_steps_english_prompt.csv", index=False)